In [0]:
import glob
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# from vgg16 import VGG16
from tqdm import tqdm_notebook as tqdm
import pickle
import pandas as pd
from keras.preprocessing import sequence
from keras.models import Sequential , load_model
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector, Activation, Flatten
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
import json
from keras.applications.inception_v3 import InceptionV3 ,preprocess_input
from keras.preprocessing import image
#from keras.applications.vgg16 import VGG16, preprocess_input

Using TensorFlow backend.


In [0]:
!pip install -q h5py pyyaml

In [0]:
#!pip install tf-nightly-gpu-2.0-preview

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#https://drive.google.com/open?id=1TCF_0YG-URDFBqxL5iEJqATqvBTkKX48
file_id = '1TCF_0YG-URDFBqxL5iEJqATqvBTkKX48'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('model.h5')

In [0]:
#https://drive.google.com/open?id=1gRgMHtLEtDZ1sXVJVBvNP5F70Il4sY4j
file_id = '1gRgMHtLEtDZ1sXVJVBvNP5F70Il4sY4j'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('captions_train2017.json')

In [0]:
#https://drive.google.com/open?id=1z9ub2AVL85D4YeZSHsism_55QPqIEZ19
file_id = '1z9ub2AVL85D4YeZSHsism_55QPqIEZ19'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('captions_val2017.json')

In [0]:
token = '/content/captions_train2017.json'
token_ = '/content/captions_val2017.json'

In [0]:
json_data = open(token)
json_data_ = open(token_)

In [0]:
captions = json.load(json_data)
captions_ = json.load(json_data_)

In [0]:
image2id = {}

for i, val in enumerate(captions['images']):
    image2id[val['id']] = val['file_name']
    
id2captions = {}

for i, val in enumerate(captions['annotations']):
    id2captions[val['image_id']] = val['caption']
    
    
image2caption = {}
for key, val in id2captions.items():
    image2caption[image2id[key]] = val

In [0]:
image2id_ = {}

for i, val in enumerate(captions_['images']):
    image2id_[val['id']] = val['file_name']
    
id2captions_ = {}

for i, val in enumerate(captions_['annotations']):
    id2captions_[val['image_id']] = val['caption']
    
image2caption_ = {}
for key, val in id2captions_.items():
    image2caption_[image2id_[key]] = val

In [0]:
#https://drive.google.com/open?id=1dUfGyOHOFWTHNaf4-b_ZoUufOFOUK7Ep
file_id = '1dUfGyOHOFWTHNaf4-b_ZoUufOFOUK7Ep'
downloaded = drive.CreateFile({'id': file_id})
#downloaded.GetContentFile('encoding_train_2017.bin')

#https://drive.google.com/open?id=14l33hIjKsZ4sOm_BQBUMQU3lmA5Rfyfx
file_id = '14l33hIjKsZ4sOm_BQBUMQU3lmA5Rfyfx'
downloaded = drive.CreateFile({'id': file_id})
#downloaded.GetContentFile('encoding_val_2017.bin')

#1TXF_KBnPFxGo2PtAnSFNoYpKBjkm6-HB
file_id = '1TXF_KBnPFxGo2PtAnSFNoYpKBjkm6-HB'
downloaded = drive.CreateFile({'id': file_id})
#downloaded.GetContentFile('encoding_test.bin')

In [0]:
#encoding_train = {}
with open('/content/encoding_train_2017.bin','rb') as f:
    encoding_train = pickle.load(f)
    
#encoding_val = {}
with open('/content/encoding_val_2017.bin','rb') as f:
    encoding_val = pickle.load(f)
       
#encoding_test = {}
with open('/content/encoding_test.bin','rb') as f:
    encoding_test = pickle.load(f)
    

In [0]:
captions = []
for key, val in image2caption.items():
    #caps.append(val)
    captions.append(' '.join(list(val.split()))) # list to string

In [0]:
for key, val in image2caption_.items():
    #caps.append(val)
    captions.append(' '.join(list(val.split()))) # list to string

In [0]:
words = [i.split() for i in captions]

In [0]:
unique = []
for i in words:
    unique.extend(i)

In [0]:
unique.extend("<start>".split()) 
unique.extend("<end>".split())
#unique.extend("<unk>".split())

In [0]:
unique = list(set(unique))

In [0]:
unique.sort()

In [0]:
word2idx = {val:index for index, val in enumerate(unique)}

In [0]:
word2idx['<start>']

558

In [0]:
idx2word = {index:val for index, val in enumerate(unique)}

In [0]:
max_len = 0
for c in captions:
    c = c.split()
    if len(c) > max_len:
        max_len = len(c)
max_len

49

In [0]:
vocab_size = len(unique)
vocab_size

25939

In [0]:
f = open('mscoco_training_dataset.txt', 'w')
f.write("image_id\tcaptions\n")

18

In [0]:
for key, val in image2caption.items():
    f.write(key + "\t" + "<start> " + val +" <end>" + "\n")

f.close()

In [0]:
f = open('mscoco_validation_dataset.txt', 'w')
f.write("image_id\tcaptions\n")

18

In [0]:
for key, val in image2caption_.items():
    f.write(key + "\t" + "<start> " + val +" <end>" + "\n")

f.close()

In [0]:
df = pd.read_csv('mscoco_training_dataset.txt', delimiter='\t')

In [0]:
df = df.dropna() # eliminate NaN value

In [0]:
df_ = pd.read_csv('mscoco_validation_dataset.txt', delimiter='\t')
df_ = df_.dropna() # eliminate NaN value

In [0]:
def data_generator(df, batch_size=32):
# Shuffling the dataframe before creating batches
#     df = df.sample(frac=1)
    c = [i for i in df['captions']]
    imgs = [i for i in df['image_id']]
    
    partial_caps = []
    images = []
    next_words = []
    
    
    count = 0
    while True:
        for i, text in enumerate(c):
            current_image = encoding_train[imgs[i]]
            for j in range(len(text.split())-1):
                count+=1
                #if text.split()[j] in word2idx:
                #    partial = [word2idx[text.split()[j]]]
                #else:
                #    partial = [word2idx['<unk>']]
                partial = [word2idx[txt] for txt in text.split()[:j+1]]
                
                partial_caps.append(partial)

                # Initializing with zeros to create a one-hot encoding matrix
                # This is what we have to predict
                # Hence initializing it with vocab_size length
                n = np.zeros(vocab_size)
                # Setting the next word to 1
                #if text.split()[j+1] in word2idx:
                #    n[word2idx[text.split()[j+1]]] = 1
                #else:
                #    n[word2idx['<unk>']] = 1
                n[word2idx[text.split()[j+1]]] = 1
                
                next_words.append(n)
                images.append(current_image)

                if count >= batch_size:
                    images = np.asarray(images)
                    next_words = np.asarray(next_words)
                    # padding 0 (default) post(backward) maxlen(set max_len and if size is over, cut from the post(back))
                    partial_caps = sequence.pad_sequences(partial_caps, maxlen=max_len, padding='post') 
                    yield [[images, partial_caps], next_words]
                    partial_caps = []
                    images = []
                    next_words = []
                    count = 0

    
    
#     partial_caps = sequence.pad_sequences(partial_caps, maxlen=max_len, padding='post')
#     return np.array(partial_caps)

In [0]:
def data_generator_val(df_, batch_size=32):
# Shuffling the dataframe before creating batches
#     df = df.sample(frac=1)
    c = [i for i in df_['captions']]
    imgs = [i for i in df_['image_id']]
    
    partial_caps = []
    images = []
    next_words = []
    
    
    count = 0
    while True:
        for i, text in enumerate(c):
            current_image = encoding_val[imgs[i]]
            for j in range(len(text.split())-1):
                count+=1
                #if text.split()[j] in word2idx:
                #    partial = [word2idx[text.split()[j]]]
                #else:
                #    partial = [word2idx['<unk>']]
                partial = [word2idx[txt] for txt in text.split()[:j+1]]
                
                partial_caps.append(partial)

                # Initializing with zeros to create a one-hot encoding matrix
                # This is what we have to predict
                # Hence initializing it with vocab_size length
                n = np.zeros(vocab_size)
                # Setting the next word to 1
                #if text.split()[j+1] in word2idx:
                #    n[word2idx[text.split()[j+1]]] = 1
                #else:
                #    n[word2idx['<unk>']] = 1
                n[word2idx[text.split()[j+1]]] = 1
                
                next_words.append(n)
                images.append(current_image)

                if count >= batch_size:
                    images = np.asarray(images)
                    next_words = np.asarray(next_words)
                    # padding 0 (default) post(backward) maxlen(set max_len and if size is over, cut from the post(back))
                    partial_caps = sequence.pad_sequences(partial_caps, maxlen=max_len, padding='post') 
                    yield [[images, partial_caps], next_words]
                    partial_caps = []
                    images = []
                    next_words = []
                    count = 0

    
    
#     partial_caps = sequence.pad_sequences(partial_caps, maxlen=max_len, padding='post')
#     return np.array(partial_caps)

In [0]:
embedding_size = 300
batch_size = 128

In [0]:
from keras.models import Model
from keras.layers import *

input_layer1 = Input(shape = (2048,))
hidden_layer1 = Dense(embedding_size, activation='relu')(input_layer1)
out1 = RepeatVector(max_len)(hidden_layer1)
image_model1 = Model(input_layer1, out1)

#image_model1.summary()

input_layer2 = Input(shape = (max_len,))

# word embedding 
# mapping "Word" to "R-dimension Vector" # 단어가 의미있는 vector로 변환 # 비슷한 단어는 비슷한 vector로 변환
emb_layer2 = Embedding(vocab_size, embedding_size, input_length=max_len)(input_layer2)


# return_sequences = true : 전체 순서열을 3차원 텐서 형태로 출력
lstm_layer2 = LSTM(256, return_sequences=True)(emb_layer2)

# 3차원 텐서 형태로 입력 받을 수 있게 확장 using TimeDistributed
out2 = TimeDistributed(Dense(300))(lstm_layer2)
caption_model1 = Model(input_layer2, out2)

#caption_model1.summary()

merged = concatenate([out1,out2], axis = 2)
x = Bidirectional(LSTM(256, return_sequences=False))(merged)
x = Dense(vocab_size)(x)

# softmax activation to get probability distribution that total sum is 1.
# called Attention Distribution and each element is called Attention Weight
out3 = Activation('softmax')(x)

final_model1 = Model([input_layer1,input_layer2], out3)



Instructions for updating:
Colocations handled automatically by placer.


In [0]:
from keras.callbacks import ModelCheckpoint

filepath = "model.h5"
final_model1.load_weights("model.h5", by_name=True)

checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min', save_weights_only = True)
callbacks_list = [checkpoint]

In [1]:
from keras.optimizers import RMSprop, Adam
#opt=Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

opt = RMSprop(lr=0.0001, decay=1e-6)

Using TensorFlow backend.


NameError: ignored

In [0]:
final_model1.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [0]:
final_model1.fit_generator(data_generator(df,batch_size=256), steps_per_epoch=2647*2, validation_data=data_generator_val(df_,batch_size = 256),validation_steps = 125,callbacks=callbacks_list ,epochs=1,verbose=1) 

Epoch 1/1
 744/5294 [===>..........................] - ETA: 20:01 - loss: 2.8657 - acc: 0.4948

KeyboardInterrupt: ignored